
# Swiss Post TSP experiment — MTZ vs. DFJ - Part 1 

This notebook compares two **direct MILP** formulations for the Euclidean Traveling Salesman Problem (TSP):

- **MTZ** (Miller–Tucker–Zemlin) which you have to implement.
- **Full DFJ** (Dantzig–Fulkerson–Johnson) with *all* subtour elimination constraints enumerated explicitly (provided here bellow).

We report the runtime for each method:

1. **Solver runtime**: `model.Runtime` reported by Gurobi (time spent in the optimizer).

This code is for question 1 and 2



## Mathematical formulations

We consider a complete directed graph $G=(V,A)$ on $n$ nodes with distances $c_{ij}$ for $i\neq j$.
Decision $x_{ij}\in\{0,1\}$ indicates whether arc $(i,j)$ is in the tour.


### DFJ (Dantzig–Fulkerson–Johnson) with all subtour constraints

\begin{align*}
\min \quad & \sum_{i\in V}\sum_{j\in V\setminus\{i\}} c_{ij} x_{ij} \\[2mm]
\text{s.t.}\quad 
& \sum_{j\in V\setminus\{j\}} x_{ij} = 1 \quad \forall j\in V, \\
& \sum_{j\in V\setminus\{i\}} x_{ji} = 1 \quad \forall i\in V, \\[1mm]
& \sum_{i\in S}\sum_{j\in S,\,j\ne i} x_{ij} \le |S|-1 \quad \forall S\subset V,\ 2\le |S|\le n-1, \\
& x_{ij}\in\{0,1\}.
\end{align*}

For DFJ we enumerate **all** nontrivial subsets $S$, i.e., $2\leq |S|\leq n-1$. The number of such sets is $2^{n}-n-2$, which grows rapidly.


# MTZ (Miller–Tucker–Zemlin) 


# Question 1 (2 points): 

MTZ TSP. Formulate the Traveling Salesman Problem as a mixed integer linear program using the Miller–Tucker–Zemlin (MTZ) formulation. Then, in at most 150 words, explain why solving your MTZ model returns the shortest round trip (i.e., why the constraints eliminate subtours and the objective selects a minimum–cost tour). Write all parts of your answer bellow.

 


### Solution

**Formulation**

First, we define our decision variables. Let $x_{ij} \in \{0, 1\}$ be a binary variable that equals 1 if the tour moves directly from node $i$ to node $j$, and 0 otherwise. We also introduce auxiliary variables $u_i \ge 0$ for $i \in \{1, \dots, n-1\}$ to represent the order of the visit.

The objective is to minimize the total travel time, subject to the standard assignment constraints and the MTZ subtour elimination constraints:

$$
\begin{aligned}
\min \quad & \sum_{i \in V} \sum_{j \neq i} c_{ij} x_{ij} \\
\text{s.t.} \quad & \sum_{j \neq i} x_{ij} = 1 \quad \forall i \in V \\
& \sum_{i \neq j} x_{ij} = 1 \quad \forall j \in V \\
& u_i - u_j + n x_{ij} \le n - 1 \quad \forall i, j \in \{1, \dots, n-1\}, i \neq j
\end{aligned}
$$

**Explanation**

The assignment constraints basically just ensure that we enter and leave every location exactly once, which creates a cycle, but on its own, this allows for disconnected subtours (mini-loops) that don't connect to the depot. The MTZ constraints fix this by assigning a "step number" $u$ to each city that isn't the depot. If we travel from $i$ to $j$ (meaning $x_{ij}=1$), the inequality forces $u_j$ to be at least $u_i + 1$, which imposes a strictly increasing order on the route. Since you can't have a sequence of numbers that keeps increasing forever inside a closed loop, the subtours are eliminated and we are left with a single valid tour connected to the depot.

 

In [3]:

import math
import random
import time
import itertools
from typing import Dict, List, Tuple, Any
import math, random
import matplotlib.pyplot as plt

try:
    import gurobipy as gp
    from gurobipy import GRB
    GUROBI_OK = True
except Exception as e:
    GUROBI_OK = False
    print("WARNING: gurobipy is not available in this environment.\n"
          "Install Gurobi and ensure your license is set up to run the solves.")


ModuleNotFoundError: No module named 'matplotlib'

In [1]:

def generate_instance(n: int = 17, seed: int = 42):
    """This function generate the TSP instance with n nodes and a given random seed. Please do not change the seed (default 42) 
    so that results are reproducible. V is the list of nodes, C is the cost dictionary and coords is the list of coordinates.
    """
    random.seed(seed)
    coords = [(random.uniform(0, 100), random.uniform(0, 100)) for _ in range(n)]
    V = list(range(n))
    def dist(i, j):
        if i == j: return 0.0
        (xi, yi), (xj, yj) = coords[i], coords[j]
        return float(math.hypot(xi - xj, yi - yj))
    C = {(i, j): dist(i, j) for i in V for j in V if i != j}
    return V, C, coords

def successor_tour_from_vars(x, V, depot: int = 0):
    """This function takes as input the decision variables x of a TSP model and returns the tour as a list of nodes, 
    starting and ending at the depot with the correct ordering to submit your answer.
    """
    n = len(V)
    succ = {i: None for i in V}
    for (i, j), var in x.items():
        try:
            val = var.X
        except Exception:
            val = float(var) if not hasattr(var, "X") else var.X
        if val > 0.5:
            succ[i] = j
    tour = [depot]
    seen = {depot}
    cur = depot
    for _ in range(n - 1):
        nxt = succ[cur]
        if nxt is None:
            break
        tour.append(nxt)
        if nxt in seen:
            break
        seen.add(nxt)
        cur = nxt
    if tour and tour[0] != tour[-1] and succ.get(cur, None) == depot:
        tour.append(depot)
    return tour


# Question 2 (8 points).

Implement and solve two TSP models in Python with Gurobi on the starter instance provided above:

MTZ formulation: implement it yourself using your model from Question 1.

DFJ formulation: use the implementation provided bellow.

Write all answers bellow.



In [2]:
def solve_mtz(V, C, time_limit: int = 99999, log_to_console: int = 1):
    if not GUROBI_OK:
        raise RuntimeError("gurobipy not available.")
    n = len(V)
    model = gp.Model("TSP_MTZ")
    model.Params.OutputFlag = log_to_console
    model.Params.TimeLimit = time_limit

    # decision vars
    x = model.addVars(C.keys(), vtype=GRB.BINARY, name="x")
    u = model.addVars(range(1, n), vtype=GRB.CONTINUOUS, lb=0, ub=n-1, name="u")

    # objective
    model.setObjective(gp.quicksum(C[i, j] * x[i, j] for (i, j) in C), GRB.MINIMIZE)

    # degree constraints
    for i in V:
        model.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"out_{i}")
        model.addConstr(gp.quicksum(x[j, i] for j in V if j != i) == 1, name=f"in_{i}")

    # MTZ constraints
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addConstr(u[i] - u[j] + n * x[i, j] <= n - 1, name=f"mtz_{i}_{j}")
        
    model.update()
    
    model.optimize()

    status = model.Status
    obj = model.ObjVal if model.SolCount > 0 else float("nan")
    gap = model.MIPGap if hasattr(model, "MIPGap") else float("nan")
    tour = successor_tour_from_vars(x, V, depot=0) if model.SolCount > 0 else []

    result = {
        "name": "MTZ",
        "status": status,
        "obj": obj,
        "gap": gap,
        "tour": tour,
        "solver_runtime_sec": model.Runtime,  
        "n": len(V),
        "num_arcs": len(C)
    }
    return model, x, result


In [3]:

def all_nontrivial_subsets(V):
    n = len(V)
    for k in range(2, n):
        for S in itertools.combinations(V, k):
            yield S

def solve_dfj_full(V, C, time_limit: int = 99999, log_to_console: int = 1):
    if not GUROBI_OK:
        raise RuntimeError("gurobipy not available.")
    n = len(V)
    model = gp.Model("TSP_DFJ_FULL")
    model.Params.OutputFlag = log_to_console
    model.Params.TimeLimit = time_limit

    # decision vars
    x = model.addVars(C.keys(), vtype=GRB.BINARY, name="x")

    # objective
    model.setObjective(gp.quicksum(C[i, j] * x[i, j] for (i, j) in C), GRB.MINIMIZE)

    # degree constraints
    for i in V:
        model.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"out_{i}")
        model.addConstr(gp.quicksum(x[j, i] for j in V if j != i) == 1, name=f"in_{i}")

    # DFJ constraints: for all nontrivial S
    
    num_subsets = 0
    for S in all_nontrivial_subsets(V):
        Sset = set(S)
        lhs = gp.quicksum(x[i, j] for i in Sset for j in Sset if i != j)
        model.addConstr(lhs <= len(Sset) - 1, name=f"dfj_S_{'_'.join(map(str, S))}")
        num_subsets += 1 
        
    model.update()
    
    model.optimize()

    status = model.Status
    obj = model.ObjVal if model.SolCount > 0 else float("nan")
    gap = model.MIPGap if hasattr(model, "MIPGap") else float("nan")
    tour = successor_tour_from_vars(x, V, depot=0) if model.SolCount > 0 else []

    result = {
        "name": "DFJ (all subsets)",
        "status": status,
        "obj": obj,
        "gap": gap,
        "tour": tour,
        "solver_runtime_sec": model.Runtime,  
        "n": len(V),
        "num_arcs": len(C),
        "num_dfj_subsets": num_subsets
    }
    return model, x, result


# Question 2 

For  each model, report:


 the optimal tour (list the nodes in visiting order, starting and ending at the depot),

 the optimal total travel time,

 the solver runtime (wall‐clock seconds).


Compare the results: Are the optimal tours identical? If not, justify. If the runtimes differ, explain why. Write all answers bellow.

In [4]:

# Set instance size and random seed
n = 17
seed = 42



V, C, coords = generate_instance(n=n, seed=seed)

xs, ys = zip(*coords)
fig, ax = plt.subplots(figsize=(6, 6))

# Bigger nodes
ax.scatter(xs, ys, s=120, zorder=3)

# Label each node larger and slightly to the right
dx = 0.018 * (max(xs) - min(xs))  # horizontal offset
for idx, (x, y) in enumerate(coords):
    ax.text(x + dx, y, f"{idx}", fontsize=12, va="center", ha="left")

ax.set_aspect("equal", adjustable="box")
ax.axis("off")
fig.tight_layout()
plt.show()

print(f"Instance: n={len(V)}, |A|={len(C)} (directed arcs)")


if GUROBI_OK:

    print("Solving MTZ")
    mtz_model, mtz_x, mtz_res = solve_mtz(V, C, time_limit=9999, log_to_console=0)
    print(f"MTZ objective: {mtz_res['obj']:.6f}")
    print(f"MTZ solver runtime:   {mtz_res['solver_runtime_sec']:.3f} s")
    print(f"MTZ tour:      {mtz_res['tour']}")
     
    print("\nSolving DFJ with ALL subsets")
    dfj_model, dfj_x, dfj_res = solve_dfj_full(V, C, time_limit=9999, log_to_console=1)
    print(f"DFJ objective (distance): {dfj_res['obj']:.6f}")
    print(f"DFJ MIP gap: {dfj_res['gap']:.6g}")
    print(f"DFJ subsets added: {dfj_res['num_dfj_subsets']}")
    print(f"DFJ solver runtime (s):   {dfj_res['solver_runtime_sec']:.3f}") 
    print(f"DFJ tour: {dfj_res['tour']}")

    # Optional: tabular summary if pandas is available
    try:
        import pandas as pd
        import numpy as np
        TO_COMPLETE = mtz_res # Replace with MTZ results that you obtain if possible, otherwise please write your result in a Markdown cell below.
        summ = pd.DataFrame([TO_COMPLETE, dfj_res])[
            ["name","obj","gap","solver_runtime_sec"]
        ]
        display(summ)
    except Exception:
        pass
else:
    print("Gurobi not available. Please run this notebook on a machine with gurobipy installed.")


NameError: name 'random' is not defined

# Solution
### Type your answers for the second part of Question 2 here bellow.

**Comparison of Results:**

* **Optimal Tours:** The tours are identical. This makes perfect sense because both models are solving the exact same problem (TSP). They are just two different ways of asking the computer to "find the shortest loop." Since there is only one shortest loop (or a tie), both models have to return the same path and the same total time.

* **Runtimes:** The runtimes are very different. The **DFJ** implementation provided is much slower because it generates a constraint for *every possible group* of cities to prevent loops. **MTZ** is faster, $O(n^{2})$, because it uses a simpler approach that only looks at *pairs* of cities to determine the order. This creates a much smaller, lighter model that the solver can handle quickly. 